# Tarefa de classificação de texto

Como mencionamos, vamos focar em uma tarefa simples de classificação de texto baseada no dataset **AG_NEWS**, que consiste em classificar manchetes de notícias em uma das 4 categorias: Mundo, Esportes, Negócios e Ciência/Tecnologia.

## O Dataset

Este dataset está integrado no módulo [`torchtext`](https://github.com/pytorch/text), então podemos acessá-lo facilmente.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Aqui, `train_dataset` e `test_dataset` contêm coleções que retornam pares de rótulo (número da classe) e texto, respectivamente, por exemplo:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Então, vamos imprimir as primeiras 10 novas manchetes do nosso conjunto de dados:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Porque conjuntos de dados são iteradores, se quisermos usar os dados várias vezes, precisamos convertê-los para lista:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenização

Agora precisamos converter o texto em **números** que podem ser representados como tensores. Se quisermos uma representação no nível de palavras, precisamos fazer duas coisas:
* usar um **tokenizador** para dividir o texto em **tokens**
* construir um **vocabulário** desses tokens.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Usando vocabulário, podemos facilmente codificar nossa string tokenizada em um conjunto de números:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Representação de texto com Bag of Words

Como as palavras representam significado, às vezes podemos entender o significado de um texto apenas observando as palavras individuais, independentemente da ordem em que aparecem na frase. Por exemplo, ao classificar notícias, palavras como *clima*, *neve* provavelmente indicam *previsão do tempo*, enquanto palavras como *ações*, *dólar* seriam associadas a *notícias financeiras*.

A representação vetorial **Bag of Words** (BoW) é a representação vetorial tradicional mais comumente usada. Cada palavra está vinculada a um índice do vetor, e o elemento do vetor contém o número de ocorrências de uma palavra em um determinado documento.

![Imagem mostrando como uma representação vetorial Bag of Words é representada na memória.](../../../../../translated_images/bag-of-words-example.606fc1738f1d7ba98a9d693e3bcd706c6e83fa7bf8221e6e90d1a206d82f2ea4.br.png) 

> **Note**: Você também pode pensar no BoW como a soma de todos os vetores one-hot-encoded para palavras individuais no texto.

Abaixo está um exemplo de como gerar uma representação Bag of Words usando a biblioteca python Scikit Learn:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Para calcular o vetor bag-of-words a partir da representação vetorial do nosso conjunto de dados AG_NEWS, podemos usar a seguinte função:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Nota:** Aqui estamos usando a variável global `vocab_size` para especificar o tamanho padrão do vocabulário. Como frequentemente o tamanho do vocabulário é bastante grande, podemos limitar o tamanho do vocabulário às palavras mais frequentes. Tente reduzir o valor de `vocab_size` e executar o código abaixo, e veja como isso afeta a precisão. Você deve esperar alguma queda na precisão, mas nada dramático, em troca de um desempenho maior.


## Treinando o classificador BoW

Agora que aprendemos como construir a representação Bag-of-Words do nosso texto, vamos treinar um classificador com base nela. Primeiro, precisamos converter nosso conjunto de dados para treinamento de forma que todas as representações vetoriais posicionais sejam transformadas em representações Bag-of-Words. Isso pode ser feito passando a função `bowify` como o parâmetro `collate_fn` para o `DataLoader` padrão do torch:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Agora vamos definir uma rede neural classificadora simples que contém uma camada linear. O tamanho do vetor de entrada é igual a `vocab_size`, e o tamanho da saída corresponde ao número de classes (4). Como estamos resolvendo uma tarefa de classificação, a função de ativação final é `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Agora vamos definir o loop de treinamento padrão do PyTorch. Como nosso conjunto de dados é bastante grande, para fins de ensino treinaremos apenas por uma época, e às vezes até por menos de uma época (especificar o parâmetro `epoch_size` nos permite limitar o treinamento). Também iremos relatar a precisão acumulada do treinamento durante o processo; a frequência de relatórios é especificada usando o parâmetro `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrams, TriGrams e N-Grams

Uma limitação da abordagem de bag of words é que algumas palavras fazem parte de expressões compostas, por exemplo, a palavra 'hot dog' tem um significado completamente diferente das palavras 'hot' e 'dog' em outros contextos. Se representarmos as palavras 'hot' e 'dog' sempre pelos mesmos vetores, isso pode confundir nosso modelo.

Para lidar com isso, as **representações N-gram** são frequentemente usadas em métodos de classificação de documentos, onde a frequência de cada palavra, bi-palavra ou tri-palavra é uma característica útil para treinar classificadores. Na representação bigram, por exemplo, adicionaremos todos os pares de palavras ao vocabulário, além das palavras originais.

Abaixo está um exemplo de como gerar uma representação de bag of words com bigram usando o Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

A principal desvantagem da abordagem N-gram é que o tamanho do vocabulário começa a crescer muito rapidamente. Na prática, precisamos combinar a representação N-gram com algumas técnicas de redução de dimensionalidade, como *embeddings*, que discutiremos na próxima unidade.

Para usar a representação N-gram em nosso conjunto de dados **AG News**, precisamos construir um vocabulário especial de ngram:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Poderíamos então usar o mesmo código acima para treinar o classificador, no entanto, isso seria muito ineficiente em termos de memória. Na próxima unidade, treinaremos um classificador de bigramas usando embeddings.

> **Nota:** Você pode deixar apenas os ngrams que aparecem no texto mais do que o número especificado de vezes. Isso garantirá que bigramas pouco frequentes sejam omitidos e reduzirá significativamente a dimensionalidade. Para fazer isso, defina o parâmetro `min_freq` para um valor mais alto e observe a mudança no tamanho do vocabulário.


## Frequência de Termos e Frequência Inversa de Documentos TF-IDF

Na representação BoW, as ocorrências de palavras são ponderadas igualmente, independentemente da palavra em si. No entanto, é evidente que palavras frequentes, como *a*, *em*, etc., são muito menos importantes para a classificação do que termos especializados. De fato, na maioria das tarefas de PLN, algumas palavras são mais relevantes do que outras.

**TF-IDF** significa **frequência de termos–frequência inversa de documentos**. É uma variação do modelo bag of words, onde, em vez de um valor binário 0/1 indicando a presença de uma palavra em um documento, utiliza-se um valor de ponto flutuante, que está relacionado à frequência de ocorrência da palavra no corpus.

Mais formalmente, o peso $w_{ij}$ de uma palavra $i$ no documento $j$ é definido como:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
onde
* $tf_{ij}$ é o número de ocorrências de $i$ em $j$, ou seja, o valor BoW que vimos anteriormente
* $N$ é o número de documentos na coleção
* $df_i$ é o número de documentos que contêm a palavra $i$ em toda a coleção

O valor TF-IDF $w_{ij}$ aumenta proporcionalmente ao número de vezes que uma palavra aparece em um documento e é ajustado pelo número de documentos no corpus que contêm a palavra, o que ajuda a compensar o fato de que algumas palavras aparecem com mais frequência do que outras. Por exemplo, se a palavra aparece em *todos* os documentos da coleção, $df_i=N$, e $w_{ij}=0$, e esses termos seriam completamente desconsiderados.

Você pode criar facilmente a vetorização TF-IDF de texto usando Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Conclusão

No entanto, embora as representações TF-IDF atribuam pesos de frequência a diferentes palavras, elas não conseguem representar significado ou ordem. Como disse o famoso linguista J. R. Firth em 1935: “O significado completo de uma palavra é sempre contextual, e nenhum estudo de significado fora do contexto pode ser levado a sério.”. Mais adiante no curso, aprenderemos como capturar informações contextuais de textos utilizando modelagem de linguagem.



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automatizadas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte autoritativa. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações equivocadas decorrentes do uso desta tradução.
